In [1]:
cd ..

/home/smaug/ownCloud/praca/reasonable-embeddings/src


In [2]:
from pathlib import Path
base_dir = Path('../local/out/elpp/')

base_dir.mkdir(parents=True, exist_ok=True)

In [3]:
from src.elpp.gen import gen
import numpy as np

rng = np.random.default_rng(seed=42)

reasoners = [gen(100, 4, rng, 200) for _ in range(40)]

 68%|██████▊   | 203/300 [00:00<00:00, 865.50it/s] 


In [4]:
import lzma
import dill

with lzma.open(base_dir / 'reasoners.dill.xz', 'wb') as f:
    dill.dump(reasoners, f)

In [5]:
rng = np.random.default_rng(seed=0xc0ffee)

test_reasoners = [gen(100, 4, rng, 200) for _ in range(20)]

 63%|██████▎   | 190/300 [00:00<00:00, 815.42it/s] 


In [6]:
with lzma.open(base_dir / 'test_reasoners.dill.xz', 'wb') as f:
    dill.dump(test_reasoners, f)
